In [1]:
import time

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torchvision.datasets import CelebA
from torch.utils.data import DataLoader

![Alt text](image-1.png)

![Alt text](image.png)

In [2]:
# Normalise data
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4, padding_mode='reflect')
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [9]:
# Batch size
batch_size_train = 128
batch_size_test = 100

# Create train and test data

train_data = CelebA(root='./../../Fifth_Year/Sem2/COMP3710/data/celeba', 
                    split='train', download=True, transform=transform_train)
test_data = CelebA(root='./../../Fifth_Year/Sem2/COMP3710/data/celeba', 
                    split='test', download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified
